In [1]:
import csv

def parse_data(f_name):
    f = open(f_name)
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f)
        yield from csv.reader(f,dialect = dialect)
    finally:
        f.close()

In [2]:
def coroutine(fn):
    def inner(*arg,**kwargs):
        g = fn(*arg,**kwargs)
        next(g)
        return g
    return inner


In [3]:
@coroutine
def save_csv(f_name):
    with open(f_name,'w',newline='') as f:
        writer = csv.writer(f)
        while True:
            row = yield
            writer.writerow(row)

In [4]:
@coroutine
def filter_data(filter_pred,target):
    while True:
        row = yield
        if filter_pred(row):
            target.send(row)
            

In [5]:
@coroutine
def pipeline_coro(out_file,name_filters):
    save = save_csv(out_file)
    
    
    target = save
    for name_filter in name_filters:
        target = filter_data(lambda d ,v =name_filter:v in d[0],target)
        
    while True:
        received = yield
        target.send(received)
        

In [6]:
from contextlib import contextmanager

In [7]:
@contextmanager
def pipeline(out_file,name_filters):
    p = pipeline_coro(out_file,name_filters)
    try:
        yield p
    finally:
        p.close()

In [8]:
with pipeline('out.csv',('Chevrolet','Carlo','Landau'))as p:
    for row in parse_data('cars.csv'):
        p.send(row)

In [9]:
with open('out.csv')as f:
    for row in f:
        print(row)

Chevrolet Monte Carlo Landau,15.5,8,350.0,170.0,4165.,11.4,77,US

Chevrolet Monte Carlo Landau,19.2,8,305.0,145.0,3425.,13.2,78,US

